In [1]:
from utils import get_path, group_cols
from datatable import dt, f

# Descrição

Neste notebook iremos formar a tabela procedimentos.

# Tabelas base

## SIH

### Colunas da tabela

In [2]:
cols_proc = [
  'ano',
  'parto_normal',
  'cnes',
  'hosp_cod_municipio',
  'cod_municipio',
]

In [3]:
path_sih = get_path('SIH', 'SIH.jay')
df_proc = dt.fread(path_sih)
df_proc = df_proc[:, cols_proc]
df_proc

,ano,parto_normal,cnes,hosp_cod_municipio,cod_municipio
,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪
0,2010,310010039,2001020,120005,120005
1,2010,310010039,2000733,120040,120040
2,2010,310010039,2000733,120040,120040
3,2010,310010039,2000733,120040,120040
4,2010,310010039,2000733,120040,120040
5,2010,310010039,2000733,120040,120040
6,2010,310010039,2000733,120040,120040
7,2010,310010039,2001020,120005,120005
8,2010,310010039,2000733,120040,120040


### Tabela agrupada

In [4]:
df_proc = group_cols(df_proc, cols_proc)
df_proc = df_proc.sort(f.ano, -f.count)
df_proc

,ano,parto_normal,cnes,hosp_cod_municipio,cod_municipio,count
,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪▪▪▪▪
0,2010,310010039,2077701,355030,355030,4358
1,2010,310010039,2077388,355030,355030,4207
2,2010,310010039,3151794,130260,130260,4145
3,2010,310010039,10480,530010,530010,4099
4,2010,310010039,2079186,355030,355030,4059
5,2010,310010039,3956369,292740,292740,3935
6,2010,310010039,26794,310620,310620,3797
7,2010,310010039,2232,280030,280030,3533
8,2010,310010039,2270609,330455,330455,3386


### Atualizando variável parto normal

In [5]:
df_proc[:, dt.update(parto_normal = f.parto_normal == 310010039)] # {310010039: 'normal', 411010034: 'cesariano'}
df_proc

,ano,parto_normal,cnes,hosp_cod_municipio,cod_municipio,count
,▪▪▪▪,▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪▪▪▪▪
0,2010,1,2077701,355030,355030,4358
1,2010,1,2077388,355030,355030,4207
2,2010,1,3151794,130260,130260,4145
3,2010,1,10480,530010,530010,4099
4,2010,1,2079186,355030,355030,4059
5,2010,1,3956369,292740,292740,3935
6,2010,1,26794,310620,310620,3797
7,2010,1,2232,280030,280030,3533
8,2010,1,2270609,330455,330455,3386


# Exportando tabela

In [6]:
output_proc = get_path('GESTANTES', 'procedimentos.csv.gzip')
df_proc.to_csv(output_proc)

In [7]:
dt.fread(output_proc)

,ano,parto_normal,cnes,hosp_cod_municipio,cod_municipio,count
,▪▪▪▪,▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪
0,2010,1,2077701,355030,355030,4358
1,2010,1,2077388,355030,355030,4207
2,2010,1,3151794,130260,130260,4145
3,2010,1,10480,530010,530010,4099
4,2010,1,2079186,355030,355030,4059
5,2010,1,3956369,292740,292740,3935
6,2010,1,26794,310620,310620,3797
7,2010,1,2232,280030,280030,3533
8,2010,1,2270609,330455,330455,3386
